# ALS Project

## History data  
- multi-entry  
- make into single entry

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

In [36]:
import string

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# load the dataset
history = pd.read_csv("data/AlsHistory.csv")

In [4]:
print(history.shape)
history.head()

(12058, 16)


subject_id  Site_of_Onset___Bulbar  Site_of_Onset___Limb  \
0          89                     NaN                   1.0   
1         329                     NaN                   1.0   
2         348                     NaN                   NaN   
3         406                     NaN                   1.0   
4         411                     1.0                   NaN   

   Site_of_Onset___Limb_and_Bulbar  Site_of_Onset___Other  \
0                              NaN                    NaN   
1                              NaN                    NaN   
2                              NaN                    NaN   
3                              NaN                    NaN   
4                              NaN                    NaN   

   Site_of_Onset___Other_Specify  Site_of_Onset___Spine  \
0                            NaN                    NaN   
1                            NaN                    NaN   
2                            NaN                    NaN   
3                            NaN                    NaN   
4                            NaN                    NaN   

   Subject_ALS_History_Delta  Disease_Duration Symptom Symptom_Other_Specify  \
0                        NaN               NaN     NaN                   NaN   
1                        0.0               NaN     NaN                   NaN   
2                        0.0               NaN     NaN                   NaN   
3                        NaN               NaN     NaN                   NaN   
4                        NaN               NaN     NaN                   NaN   

  Location  Location_Other_Specify Site_of_Onset  Onset_Delta  Diagnosis_Delta  
0      NaN                     NaN           NaN          NaN              NaN  
1      NaN                     NaN           NaN      -1706.0          -1068.0  
2      NaN                     NaN  Onset: Other       -501.0           -302.0  
3      NaN                     NaN           NaN          NaN              NaN  
4      NaN                     NaN           NaN          NaN              NaN

In [5]:
dups(history)

NameError: name 'dups' is not defined

In [6]:
# delete 'Site_of_Onset___Other_Specify' - empty
history.drop('Site_of_Onset___Other_Specify', axis=1, inplace=True)

In [7]:
# delete 'Location_Other_Specify' - empty
history.drop('Location_Other_Specify', axis=1, inplace=True)

In [8]:
# delete 'Disease_Duration' - empty
history.drop('Disease_Duration', axis=1, inplace=True)

In [9]:
# delete 'Subject_ALS_History_Delta' - empty
history.drop('Subject_ALS_History_Delta', axis=1, inplace=True)

In [10]:
print(history.shape)
history.head()

(12058, 12)


subject_id  Site_of_Onset___Bulbar  Site_of_Onset___Limb  \
0          89                     NaN                   1.0   
1         329                     NaN                   1.0   
2         348                     NaN                   NaN   
3         406                     NaN                   1.0   
4         411                     1.0                   NaN   

   Site_of_Onset___Limb_and_Bulbar  Site_of_Onset___Other  \
0                              NaN                    NaN   
1                              NaN                    NaN   
2                              NaN                    NaN   
3                              NaN                    NaN   
4                              NaN                    NaN   

   Site_of_Onset___Spine Symptom Symptom_Other_Specify Location Site_of_Onset  \
0                    NaN     NaN                   NaN      NaN           NaN   
1                    NaN     NaN                   NaN      NaN           NaN   
2                    NaN     NaN                   NaN      NaN  Onset: Other   
3                    NaN     NaN                   NaN      NaN           NaN   
4                    NaN     NaN                   NaN      NaN           NaN   

   Onset_Delta  Diagnosis_Delta  
0          NaN              NaN  
1      -1706.0          -1068.0  
2       -501.0           -302.0  
3          NaN              NaN  
4          NaN              NaN

**Split data - onset and daignosis dates from body location**  
- dates

In [11]:
history_onset_dates = history[['subject_id', 'Onset_Delta', 'Diagnosis_Delta']].copy()

In [12]:
print(history_onset_dates.shape)
history_onset_dates.head()

(12058, 3)


subject_id  Onset_Delta  Diagnosis_Delta
0          89          NaN              NaN
1         329      -1706.0          -1068.0
2         348       -501.0           -302.0
3         406          NaN              NaN
4         411          NaN              NaN

In [13]:
# remove duplicates
dups(history_onset_dates)
history_onset_dates.drop_duplicates(inplace=True)

NameError: name 'dups' is not defined

In [14]:
print(history_onset_dates.shape)
history_onset_dates.head()

(12058, 3)


subject_id  Onset_Delta  Diagnosis_Delta
0          89          NaN              NaN
1         329      -1706.0          -1068.0
2         348       -501.0           -302.0
3         406          NaN              NaN
4         411          NaN              NaN

In [15]:
onset = history_onset_dates[[ 'subject_id', 'Onset_Delta', 'Diagnosis_Delta']].copy()

In [16]:
onset.head()

subject_id  Onset_Delta  Diagnosis_Delta
0          89          NaN              NaN
1         329      -1706.0          -1068.0
2         348       -501.0           -302.0
3         406          NaN              NaN
4         411          NaN              NaN

In [17]:
# delete the empty rows
onset.dropna(axis=0, inplace=True, thresh=2)

In [18]:
onset.head(10)

subject_id  Onset_Delta  Diagnosis_Delta
1          329      -1706.0          -1068.0
2          348       -501.0           -302.0
5          533      -1023.0            -44.0
6          586       -715.0           -507.0
7          649       -341.0              NaN
9          649       -341.0              NaN
10         708      -1070.0           -986.0
11         902       -514.0            -49.0
13        1064       -550.0           -462.0
14        1234      -1181.0              NaN

In [19]:
filename = 'clean_data/onset_date.csv'
onset.to_csv(filename, index=False)

#### History location

Columns ready for analysis = 
- Site_of_Onset___Bulbar
- Site_of_Onset___Limb
- Site_of_Onset___Limb_and_Bulbar
- Site_of_Onset___Other
- Site_of_Onset___Spine
- Site_of_Onset
... can remove the columns above.


In [66]:
history_location = history[['subject_id', 'Site_of_Onset___Bulbar', 
        'Site_of_Onset___Limb', 'Site_of_Onset___Limb_and_Bulbar', 'Site_of_Onset___Other',
        'Site_of_Onset___Spine', 'Symptom', 'Symptom_Other_Specify', 'Location', 
        'Site_of_Onset',]].copy()

In [67]:
print(history_location.shape)
history_location.head()

(12058, 10)


subject_id  Site_of_Onset___Bulbar  Site_of_Onset___Limb  \
0          89                     NaN                   1.0   
1         329                     NaN                   1.0   
2         348                     NaN                   NaN   
3         406                     NaN                   1.0   
4         411                     1.0                   NaN   

   Site_of_Onset___Limb_and_Bulbar  Site_of_Onset___Other  \
0                              NaN                    NaN   
1                              NaN                    NaN   
2                              NaN                    NaN   
3                              NaN                    NaN   
4                              NaN                    NaN   

   Site_of_Onset___Spine Symptom Symptom_Other_Specify Location Site_of_Onset  
0                    NaN     NaN                   NaN      NaN           NaN  
1                    NaN     NaN                   NaN      NaN           NaN  
2                    NaN     NaN                   NaN      NaN  Onset: Other  
3                    NaN     NaN                   NaN      NaN           NaN  
4                    NaN     NaN                   NaN      NaN           NaN

In [68]:
history_location['Site_of_Onset'].unique()

array([nan, 'Onset: Other', 'Onset: Bulbar', 'Onset: Limb',
       'Onset: Spine', 'Onset: Limb and Bulbar'], dtype=object)

In [69]:
# correct site of onset columns <- 'Site_of_Onset'
def bulbar(row):
    if row['Site_of_Onset___Bulbar'] == 1:
        return 1
    elif row['Site_of_Onset']== 'Onset: Bulbar':
        return 1
    else: 
        return 0

# correct site of onset columns <- 'Site_of_Onset'
def limb(row):
    if row['Site_of_Onset___Limb'] == 1:
        return 1
    elif row['Site_of_Onset']== 'Onset: Limb':
        return 1
    else: 
        return 0

# correct site of onset columns <- 'Site_of_Onset'
def spine(row):
    if row['Site_of_Onset___Spine'] == 1:
        return 1
    elif row['Site_of_Onset']== 'Onset: Spine':
        return 1
    else: 
        return 0

# correct site of onset columns <- 'Site_of_Onset'
def limb_bulbar(row):
    if row['Site_of_Onset___Limb_and_Bulbar'] == 1:
        return 1
    elif row['Site_of_Onset']== 'Onset: Limb and Bulbar':
        return 1
    else: 
        return 0
    
# correct site of onset columns <- 'Site_of_Onset'
def other(row):
    if row['Site_of_Onset___Other'] == 1:
        return 1
    elif row['Site_of_Onset']== 'Onset: Other':
        return 1
    else: 
        return 0

In [70]:
# make 1 and 0 columns with site locations
history_location['site_onset_bulbar'] = history_location.apply(bulbar, axis=1)
history_location['site_onset_limb'] = history_location.apply(limb, axis=1)
history_location['site_onset_spine'] = history_location.apply(spine, axis=1)
history_location['site_onset_other'] = history_location.apply(other, axis=1)
history_location['site_onset_limb_and_bulbar'] = history_location.apply(limb_bulbar, axis=1)

In [71]:
print(history_location.shape)
history_location.head()

(12058, 15)


subject_id  Site_of_Onset___Bulbar  Site_of_Onset___Limb  \
0          89                     NaN                   1.0   
1         329                     NaN                   1.0   
2         348                     NaN                   NaN   
3         406                     NaN                   1.0   
4         411                     1.0                   NaN   

   Site_of_Onset___Limb_and_Bulbar  Site_of_Onset___Other  \
0                              NaN                    NaN   
1                              NaN                    NaN   
2                              NaN                    NaN   
3                              NaN                    NaN   
4                              NaN                    NaN   

   Site_of_Onset___Spine Symptom Symptom_Other_Specify Location Site_of_Onset  \
0                    NaN     NaN                   NaN      NaN           NaN   
1                    NaN     NaN                   NaN      NaN           NaN   
2                    NaN     NaN                   NaN      NaN  Onset: Other   
3                    NaN     NaN                   NaN      NaN           NaN   
4                    NaN     NaN                   NaN      NaN           NaN   

   site_onset_bulbar  site_onset_limb  site_onset_spine  site_onset_other  \
0                  0                1                 0                 0   
1                  0                1                 0                 0   
2                  0                0                 0                 1   
3                  0                1                 0                 0   
4                  1                0                 0                 0   

   site_onset_limb_and_bulbar  
0                           0  
1                           0  
2                           0  
3                           0  
4                           0

### Separate locations

In [99]:
# separate location columns
history_symptom = history_location[['subject_id','Symptom', 
                                    'Symptom_Other_Specify', 'Location']].copy()

In [100]:
print(history_symptom.shape)
history_symptom.head(20)

(12058, 4)


subject_id     Symptom Symptom_Other_Specify                  Location
0           89         NaN                   NaN                       NaN
1          329         NaN                   NaN                       NaN
2          348         NaN                   NaN                       NaN
3          406         NaN                   NaN                       NaN
4          411         NaN                   NaN                       NaN
5          533         NaN                   NaN                       NaN
6          586         NaN                   NaN                       NaN
7          649    WEAKNESS                   NaN                    FACIAL
8          649         NaN                   NaN                       NaN
9          649      Speech                   NaN                    SPEECH
10         708         NaN                   NaN                       NaN
11         902         NaN                   NaN                       NaN
12        1025         NaN                   NaN                       NaN
13        1064         NaN                   NaN                       NaN
14        1234      Speech                   NaN           TONGUE WEAKNESS
15        1234         NaN                   NaN                       NaN
16        1234  Swallowing                   NaN  ORAL PNARYNGIAL WEAKNESS
17        1309         NaN                   NaN                       NaN
18        1312         NaN                   NaN                       NaN
19        1736         NaN                   NaN                       NaN

In [101]:
# delete the empty rows
history_symptom.dropna(axis=0, inplace=True, thresh=3)
history_symptom.reset_index(inplace=True, drop=True)

In [102]:
print(history_symptom.shape)
history_symptom.head(10)

(2653, 4)


subject_id     Symptom Symptom_Other_Specify                  Location
0         649    WEAKNESS                   NaN                    FACIAL
1         649      Speech                   NaN                    SPEECH
2        1234      Speech                   NaN           TONGUE WEAKNESS
3        1234  Swallowing                   NaN  ORAL PNARYNGIAL WEAKNESS
4        2540    WEAKNESS                   NaN                 RIGHT LEG
5        2956    WEAKNESS                   NaN                  LEFT LEG
6        3085      Speech                   NaN                    BULBAR
7        3551    WEAKNESS                   NaN                     R ARM
8        3809    WEAKNESS                   NaN           (R) HAND + FOOT
9        3870    WEAKNESS                   NaN              ORAL MUSCLES

In [103]:
def make_lower(val):
    w=val.lower()
    return w

In [104]:
# make lowercase location column
history_symptom['location'] = history_symptom['Location'].apply(make_lower)
# make lowercase symptom column
history_symptom['symptom'] = history_symptom['Symptom'].apply(make_lower)

In [105]:
history_symptom.head()

subject_id     Symptom Symptom_Other_Specify                  Location  \
0         649    WEAKNESS                   NaN                    FACIAL   
1         649      Speech                   NaN                    SPEECH   
2        1234      Speech                   NaN           TONGUE WEAKNESS   
3        1234  Swallowing                   NaN  ORAL PNARYNGIAL WEAKNESS   
4        2540    WEAKNESS                   NaN                 RIGHT LEG   

                   location  
0                    facial  
1                    speech  
2           tongue weakness  
3  oral pnaryngial weakness  
4                 right leg

In [106]:
# function development
val = "1st 2 fingers-hands"
punct = set(string.punctuation)
clean_val = val.translate(str.maketrans('', '', string.punctuation))
words = clean_val.split()
print(words)

['1st', '2', 'fingershands']


In [107]:
def word_assoc(val, cat):
    '''
    takes in a string, cleans and splits it, then determines if word in a list
    val:  string
    cat:  list of words or letter to group
    '''
    punct = set(string.punctuation)
    clean_val = val.translate(str.maketrans('', '', string.punctuation))
    words = clean_val.split()
    for word in words:
        if word in cat:
            return 1
    return 0

In [108]:
# run the side functions

def side_right(val):
    rights = ['r', 'rt', 'right', 'handsright', 'righ', 'rue', 'armright', 'limbsright',
         'handsr', 'armr', 'lr', 'rleright', 'rl', 'handright', 'rleg', 'rfoot']
    return word_assoc(val, rights)

def side_left(val):
    lefts = ['l', 'lt' , 'left', 'legleft', 'lleleft', 'lr', 'handleft', 'rl', 'handl',
        'limbleft', 'footleft', 'lshoulder']
    return word_assoc(val, lefts)

def side_bilaterals(val):
    bilaterals = ['bilateral', 'legsbilateral', 'bilaterally', 'bilat', 'extremitiesbilateral',
             'handsbilateral', 'bil', 'bilateralhands']
    if word_assoc(val, bilaterals) == 1:
        return 1
    return side_left(val) and side_right(val)

In [109]:
# side of body of onset
history_symptom['right_side'] = history_symptom['location'].apply(right_side)
history_symptom['left_side'] = history_symptom['location'].apply(left_side)
history_symptom['bilateral_side'] = history_symptom['location'].apply(left_side)

In [110]:
print(history_symptom.shape)
history_symptom.head()

(2653, 8)


subject_id     Symptom Symptom_Other_Specify                  Location  \
0         649    WEAKNESS                   NaN                    FACIAL   
1         649      Speech                   NaN                    SPEECH   
2        1234      Speech                   NaN           TONGUE WEAKNESS   
3        1234  Swallowing                   NaN  ORAL PNARYNGIAL WEAKNESS   
4        2540    WEAKNESS                   NaN                 RIGHT LEG   

                   location  right_side  left_side  bilateral_side  
0                    facial           0          0               0  
1                    speech           0          0               0  
2           tongue weakness           0          0               0  
3  oral pnaryngial weakness           0          0               0  
4                 right leg           1          0               0

In [112]:
# body part of onset symptom
def loc_general(val):
    generals = ['generalized', 'body', 'general', 'global', 'muscle atrophy']
    return word_assoc(val, generals)

def loc_trunk(val):
    trunks = [ 'back', 'trunkfasciculation', 'abdominal', 'abdomen', 'armlegchest' ]
    return word_assoc(val, trunks)

def loc_face(val):
    faces = ['face', 'facial', 'tonguefacespeech', 'armsface', 'nose', 'orofacial', 
         'tongueface', 'facetongue']
    return word_assoc(val, faces)

def loc_mouth(val):
    mouths = ['mouth', 'tongue', 'oral', 'pharynx', 'pnaryngial', 'throat', 'laryngeal', 
          'dysphagia', 'larynx', 'oropharyngal', 'mouththroat', 'swallow', 'pharyngeal',
          'throatswallowing', 'oropharyngeal', 'esphagus', 'choking', 'mouthtongue',
          'legmouthafter', 'tonguelarynx', 'nasopharynx', 'thracic', 'tonguelimbs',
          'tonguefacespeech', 'anaephanyrox', 'tonguethroat', 'pharygealchokingspeech',
         'jaw', 'swallowing', 'oralpharynx', 'tongueface', 'tongueslowing', 'facetongue',
         'mouththroat', 'deglutition', 'handmouth', 'masseter', 'oropharynx']
    return word_assoc(val, mouths)

def loc_speech(val):
    speechs = ['speech', 'bulbar', 'voice', 'voicethroat', 'dysarthria', 'dysphornia',
          'bulbaire', 'pseudobulbar', 'speak', 'speechbulbar', 'dysurthria', 'dysartria',
          'dysphonia', 'speechoral', 'pronounce', 'throatvoice', 'vocal', 'speechthroat',
          'tonguemouth', 'dysparthria', 'pronunciation', 'tonguefacespeech', 
          'speechinitial', 'bulbartongue', 'dysathria', 'dysarythria', 'slurring',
          'dysarthmia', 'slurred', 'voicethroatbulbar', 'dysphagiadysarthria',
          'bulbarthroat', 'dysarthrophonia', 'slurredmouth', 'dysphagiadysarthria',
          'hoarseness', 'bulbardysarthria', 'dysarthrisa']
    return word_assoc(val, speechs)

def loc_speech_or_mouth(val):
    return loc_speech(val) or loc_mouth(val)

def loc_feet(val):
    feet = ['foot', 'feet', 'toe', 'legfoot', 'footdrop', 'feetankle', 'bublar', 'legfoot',
       'plantar', 'legsfeet', 'footleft', 'legfoot', 'rfoot']
    return word_assoc(val, feet)

def loc_legs_or_feet(val):
    # leg cat includes feet
    legs = ['lower','leg', 'legs', 'ankle', 'hip', 'le', 'lower', 'legmild', 'handlegs',
       'tibial', 'calf', 'thigh', 'ankles', 'armslegs', 'legstrunk', 'rle', 
       'knee', 'tibiales', 'tibialus', 'legleft', 'lowr', 'legsbilateral', 'legfoot',
       'armslegs', 'tonguelimbs', 'legsbilateral', 'armleg', 'feetankle', 'legproximaldistal',
       'diffuselyrllegs', 'thighs', 'hipleft', 'calves', 'calf', 'legproximal', 'legl',
       'fibuleos', 'ankledrop', '1996legs', 'legfoot', 'rleg', 'legsfeet', 'peroneal', 'rleg',
       'legfoot', 'armlegchest']
    return word_assoc(val, legs) or loc_feet(val)

def loc_hands(val):
    hands = ['finger', 'fingers', 'hands', 'hamd', 'hand', 'grip', 'thumb', 'handlegs',
        'handright', 'handsright', 'fingershands', 'handpresis', 'lthumb', 'thumbleft',
        'handdexterity', 'handarmshoulder', 'handsr', 'thenar', 'wrist', 'band',
        'extremityhand', 'handvery', 'thumbhand', 'handfingers', 'thumbs', 'handfingerthumb',
        'midfinger', 'handright', 'handleft', 'armshands', 'handsbilateral', 'handmouth',
        'handl', 'handr', 'bilateralhands', 'interossei']
    return word_assoc(val, hands)

def loc_arms_or_hands(val):
    # arms includes hands
    arms = ['arm', 'arms', 'upper', 'forearm', 'handarm', 'armsneck', 'armslegs',
       'armproximal', 'armmild', 'rue', 'armright', 'armsreduced', 'handarmshoulder', 
       'armslegs', 'tonguelimbs', 'armsface', 'uele', 'armleg', 'armshoulder',
       'elbow', 'ue', 'armbilaterally', 'forearmshoulder', 'bicepts', 'armshands', 'biceps',
       'armlegchest', 'shoulder', 'handarmshoulder', 'shoulders', 'armshoulder', 
        'forearmshoulder', 'lshoulder', 'membres superieurs']
    return word_assoc(val, arms) or loc_hands(val)

def loc_arms_and_legs(val):
    arms_and_legs = [ 'all 4 limbs', 'four limbs', '4 limbs', 'all four limbs', 'all limbs',
                'all extremities', 'in all extremities']
    if val in arms_and_legs:
        return 1
    return word_assoc(val, arms) and word_assoc(val, leg)

def loc_respiratory(val):
    respiratory = ['respiratory', 'diaphragm']
    return word_assoc(val, respiratory)

def loc_neck_head(val):
    neck_head = ['neck', 'head', 'cervical']
    return word_assoc(val, neck_head)

In [113]:
# create new dummy columns based on symptom location
history_symptom['loc_general'] = history_symptom['location'].apply(loc_general)
history_symptom['loc_trunk'] = history_symptom['location'].apply(loc_trunk)
history_symptom['loc_face'] = history_symptom['location'].apply(loc_face)
history_symptom['loc_mouth'] = history_symptom['location'].apply(loc_mouth)
history_symptom['loc_speech'] = history_symptom['location'].apply(loc_speech)
history_symptom['loc_speech_or_mouth'] = history_symptom['location'].apply(loc_speech_or_mouth)
history_symptom['loc_feet'] = history_symptom['location'].apply(loc_feet)
history_symptom['loc_legs_or_feet'] = history_symptom['location'].apply(loc_legs_or_feet)
history_symptom['loc_hands'] = history_symptom['location'].apply(loc_hands)
history_symptom['loc_arms_or_hands'] = history_symptom['location'].apply(loc_arms_or_hands)
history_symptom['loc_arms_and_legs'] = history_symptom['location'].apply(loc_arms_and_legs)
history_symptom['loc_respiratory'] = history_symptom['location'].apply(loc_respiratory)
history_symptom['loc_neck_head'] = history_symptom['location'].apply(loc_neck_head)

In [119]:
print(history_symptom.shape)
history_symptom.head(20)

(2653, 22)


subject_id     Symptom Symptom_Other_Specify  \
0          649    WEAKNESS                   NaN   
1          649      Speech                   NaN   
2         1234      Speech                   NaN   
3         1234  Swallowing                   NaN   
4         2540    WEAKNESS                   NaN   
5         2956    WEAKNESS                   NaN   
6         3085      Speech                   NaN   
7         3551    WEAKNESS                   NaN   
8         3809    WEAKNESS                   NaN   
9         3870    WEAKNESS                   NaN   
10        4390    WEAKNESS                   NaN   
11        4772    WEAKNESS                   NaN   
12        5092    WEAKNESS                   NaN   
13        5155    WEAKNESS                   NaN   
14        5588    WEAKNESS                   NaN   
15        5936      Speech                   NaN   
16        5936       OTHER            sialorrhea   
17        6502    WEAKNESS                   NaN   
18        6747    WEAKNESS                   NaN   
19        7399  Swallowing                   NaN   

                                  Location  \
0                                   FACIAL   
1                                   SPEECH   
2                          TONGUE WEAKNESS   
3                 ORAL PNARYNGIAL WEAKNESS   
4                                RIGHT LEG   
5                                 LEFT LEG   
6                                   BULBAR   
7                                    R ARM   
8                          (R) HAND + FOOT   
9                             ORAL MUSCLES   
10                   FINGERS OF BOTH HANDS   
11                                LEFT LEG   
12                             TONGUE ARMS   
13                          LEFT FOOT DROP   
14              RIGHT THUMB & INDEX FINGER   
15                                   MOUTH   
16                                   MOUTH   
17               PROXIMAL LOWER RIGHT LIMB   
18  (R) FOOT DROP FEW MONTHS LATER (L) FT.   
19                                  THROAT   

                                  location  right_side  left_side  \
0                                   facial           0          0   
1                                   speech           0          0   
2                          tongue weakness           0          0   
3                 oral pnaryngial weakness           0          0   
4                                right leg           1          0   
5                                 left leg           0          1   
6                                   bulbar           0          0   
7                                    r arm           1          0   
8                          (r) hand + foot           1          0   
9                             oral muscles           0          0   
10                   fingers of both hands           0          0   
11                                left leg           0          1   
12                             tongue arms           0          0   
13                          left foot drop           0          1   
14              right thumb & index finger           1          0   
15                                   mouth           0          0   
16                                   mouth           0          0   
17               proximal lower right limb           1          0   
18  (r) foot drop few months later (l) ft.           1          1   
19                                  throat           0          0   

    bilateral_side  loc_general  loc_trunk  ...  loc_speech  \
0                0            0          0  ...           0   
1                0            0          0  ...           1   
2                0            0          0  ...           0   
3                0            0          0  ...           0   
4                0            0          0  ...           0   
5                1            0          0  ...           0   
6                0            0          0  ...      

##

In [122]:
# how many values of each type in the symptom column
history_symptom.symptom.value_counts()
# keep 'other' values listed as 'other'...too sparce

weakness           1978
speech              293
cramps               67
fasciculations       65
atrophy              54
other                51
swallowing           47
stiffness            40
sensory_changes      35
gait_changes         23
Name: symptom, dtype: int64

In [ ]:
# make dummy columns